In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
complaints  = pd.read_csv('../final/complaints.csv')
complaints.columns
# complaints: time series? spatial info?